# **Project: Airlines Traffic Passenger Analysis**

**The main ideas of this project are**
* to improve my skills in data analysis
* create an interesting project
* draw conclusions about my findings
* to create a project for the portfolio

**I choose this topic because:**
* I don't travel much, so I want to start somewhere
* I want to work with data that has geographical information
* this problem is on Kaagle, so why not


#                               Table Of Content

<ul>
<li>
    <a href="#intro">Introduction</a>
    <ul>
        <li><a href="#data_structure">Data Structure</a></li>
        <li><a href="#questions">Questions</a></li>
    </ul>
</li>
<li>
    <a href="#data_preparation">Data preparation</a>
    <ul>
        <li><a href="#general-properties">General Properties</a></li>
        <li><a href="#data_cleaning">Data Cleaning</a></li>
    </ul>
</li>
<li>
    <a href="#eda">Exploratory Data Analysis</a>
    <ul>
        <li>
            <a href="#domorint">Domestic vs International(Q1)</a>
        </li>
        <li>
            <a href="#productive-year">The most productive year(Q2)</a>
        </li>
        <li>
            <a href="#ticket-cost">Cheap ticket or expensive?(Q3)</a>
        </li>
    </ul>
</li>
<li><a href="#conclusions">Conclusion</a></li>
</ul>

<a id='intro'></a>
# Introduction

This analysis is to examine passenger traffic data. This dataset contains information on air traffic passenger statistics by the airline and contains more than 15 thousand records. It includes information on the airlines, airports, and regions that the flights departed from and arrived at. It also includes information on the type of activity, price category, terminal, boarding area, and number of passengers. More information about dataset here: [Air_Traffic_Passenger_Statistics](http://www.kaggle.com/datasets/thedevastator/airlines-traffic-passenger-statistics)

<a id='data_structure'></a>

### Data Structure

Database consists of the following columns: 
* **Index** - element id (chr)
* **Activity Period** - the date of the activity (dbl)
* **Operating Airline** - the airline that operated the flight (chr)
* **Operating Airline IATA Code** - the IATA code of the airline that operated the flight (chr)
* **Published Airline** - The airline that published the fare for the flight (chr)
* **Published Airline IATA Code** - The IATA code of the airline that published the fare for the flight (chr)
* **GEO Summary** - A summary of the geographic region (chr)
* **GEO Region** - The geographic region (chr)
* **Activity Type Code** - The type of activity (chr)
* **Price Category Code** - The price category of the fare (chr)
* **Terminal** - The terminal of the flight (chr)
* **Boarding Area** - The boarding area of the flight (chr)
* **Passenger Count** - The number of passengers on the flight (dbl)
* **Adjusted Activity Type Code** - The type of activity, adjusted for missing data (chr)
* **Adjusted Passenger Count** - The number of passengers on the flight, adjusted for missing data (dbl)
* **Year** - The year of the activity (dbl)
* **Month** - The month of the activity (chr)

https://prod-uk-a.online.tableau.com/#/site/denysyefimov/workbooks/267615/views

In [ ]:
library(tidyverse); # metapackage of all tidyverse packages
library(dplyr); # metapackage of all dplyr packages
library(lubridate); # metapackage of all lubridate packages
library(scales); # metapackage of all scales packages
passenger_traffic_data <- read_csv("/kaggle/input/airtrafficdata/Air_Traffic_Passenger_Statistics.csv"); # loading library
colnames(passenger_traffic_data)[12] = "boarding_area"

In [ ]:
str(passenger_traffic_data) #showing data structure with details

<a id='questions'></a>

### Questions

The main questions I need to answer are:

1. Which refs are the most popular: domestic or international? What is the number of passengers for domestic refs and for international flights?
2. Which year is the most productive in terms of passenger traffic and flights? What is the worst year on the contrary?
3. Do people choose cheap tickets or expensive ones? What can it depend on?

<a id='data_preparation'></a>
# Data Preparation

In this section, I will review the structures, column types in more detail, see if there are any problem areas in the data and prepare the data for the next step. Also we're gonna explore in a deeper lever in a way that maybe we can formulate more questions.

<a id='general-properties'></a>
### General Properties

Here we're gonna explore our dataset properties checking for:
- View the unique values of each column
- What kind of variables we need to:
  - convert the data type
  - drop from the dataset
- Check for duplicates
- Check for white spaces
- Check for weird values (outliers)
- Gather more information about a specific variable
- Check if we need to create more columns with usefull data for the exploration or combine some columns in one


In [ ]:
# count of duplicated records
sum(duplicated(passenger_traffic_data)) 

In [ ]:
# checking unique values on some columns

passenger_traffic_data %>% drop_na() %>% distinct(geo_summary)

passenger_traffic_data %>% drop_na() %>% distinct(price_category_code)

passenger_traffic_data %>% drop_na() %>% distinct(geo_region)

passenger_traffic_data %>% drop_na() %>% distinct(activity_type_code)


So, we see that there is not a single duplicate line. Also, if you look at the string activity_period, year, month, you can see that activity_period = year + month. Therefore, we can delete year and month columns.

<a id='data_cleaning'></a>
### Data Cleaning

Let's start with deleting columns "year" and "month". And also let's change the type of column "activity_period" from num on date. We can combine year + month, convert it to date format and delete the activity_period column, but it takes one step more than deleting date and year and converting activity_period to date format. Also the index column has the chr type, so we need to convert it to the integer type. And let's remove unnecessary columns that are not needed in the analysis. I will also use the Adjusted Passenger Count column instead of the Passenger Count, because this column already accounts for missing data. This is worth keeping in mind when analyzing and as a result. 

In [ ]:
passenger_traffic_data_validated <- subset(passenger_traffic_data, select = -c(month, year)) # removing month and year columns
passenger_traffic_data_validated$activity_period <- ym(passenger_traffic_data_validated$activity_period) # changing type for activity_period column
passenger_traffic_data_validated$index <- as.integer(passenger_traffic_data_validated$index) # changing type for index from chr to int
df <- subset(passenger_traffic_data_validated, select = -c(operating_airline_iata_code, published_airline_iata_code, passenger_count, published_airline, published_airline_iata_code, activity_type_code, terminal, boarding_area))
str(df)

We can already draw certain conclusions. The data have already been prepared in a certain way but we were able to improve them in some ways: 
* no duplicates were found
* there are no empty cells (some studies were carried out in bigQuery because it is easier and faster to do it, and the functionality of Kaagle Notebook does not support SQL queries), that is, we do not lose data, but also, we cannot be sure that we have not lost them when we have already taken the finished database 
* we changed the types for some columns
* removed some columns that will not be useful in the research.


<a id='eda'></a>
# Exploratory Data Analysis

Now let's move closer to the analysis itself. We will go step by step through each question, build graphs and look at the results.

<a id='domorint'></a>

### Domestic vs International

The issue is not fully understood. The first thing that comes to my mind is the total volume of passengers for domestic and international type. But also, we can investigate this issue for each year, which will not be a mistake and will make sense. Therefore, we will draw conclusions for each year, and in general for the period from 2005 to 2017.

For this question, we need the column "geo_summary" which has the value Domestic and International for each airline, which shows whether it was a flight within the same country or between different countries. Also, for more detailed information, we need a column with the date period "activity_period", which shows in which month it was made.

First, let's build a pie chart that will show information for each type of flight to see the difference between them.

In [ ]:
#getting passengers summary and groping by geo_summary
df_pie_chart_data <- df %>%
    drop_na() %>%
    group_by(geo_summary) %>%
    summarise(adjusted_passenger_count = sum(adjusted_passenger_count))

df_pie_chart_data

piepercent<- round(100*df_pie_chart_data$adjusted_passenger_count/sum(df_pie_chart_data$adjusted_passenger_count), 1)

#creating pie chart
pie(df_pie_chart_data$adjusted_passenger_count, labels=paste(df_pie_chart_data$geo_summary, "(" , piepercent, "%)", sep=""), 
    main = "Domestic vs International in general(by passengers)", 
    col = rainbow(length(df_pie_chart_data$geo_summary)))

Therefore, we can see a huge advantage of domestic flights over international. So for *Domestic* we have 339042566 passengers and it is 77% percent of the total number of passengers. For *International* we have 101093590 passengers and 23% of the total number.

Now let's look at the statistics for each year, in order to see more detailed difference and difference for each type of flight for each year.

In [ ]:
options(scipen = 999)
#getting passengers summary and groping by geo_summary, year
df_line_chart_data <- df %>%
    drop_na() %>%
    group_by(activity_period = year(activity_period), geo_summary) %>%
    summarise(adjusted_passenger_count = sum(adjusted_passenger_count))

df_line_chart_data

#building line chart
ggplot(data=df_line_chart_data) + geom_line(aes(x=activity_period, 
                y=adjusted_passenger_count, group=geo_summary)) + scale_y_continuous(labels = scales::label_number_si())
                labs(title = "Domestic vs International(by year)",
                x = "Year", y = "Passengers Count")

From these data we can see that the number of flights both domestic and international is growing every year. Also, we can see one problem with the data itself. If we look at 2016, we can see that the line drops sharply. This is because for 2016 we have only 371 records in the table, and for example, for the previous year 2015 we have 1460 records. This is not very good, because taking into account this, we can draw wrong conclusions. Therefore, you need to keep this in mind.

From these two charts, you can be sure that domestic flights are much more popular than international.


<a id='productive-year'></a>
### The most productive year

So, now, let's take a closer look at the statistics for each year. It is necessary to investigate which year was the most profitable and the most relevant in terms of passenger traffic, and which was vice versa.

First of all, let's examine how many entries we have in the table for each year. This is necessary to begin with, so that we can draw certain conclusions at once and build on them.

In [ ]:
options(scipen = 999)
#getting year for bar chart
df_bar_chart_data <- df %>%
    drop_na() %>%
    group_by(activity_period = year(activity_period))

ggplot(data=df_bar_chart_data) + geom_bar(mapping=aes(x=activity_period, color=activity_period)) + 
                labs(title = "Traffic for each Year(by records)",
                x = "Year", y = "Records Count")

#getting year for line chart
df_line_chart_data <- df %>%
    drop_na() %>%
    group_by(activity_period = year(activity_period)) %>%
    summarise(adjusted_passenger_count = sum(adjusted_passenger_count))

ggplot(data=df_line_chart_data) + geom_line(aes(x=activity_period, 
                y=adjusted_passenger_count), stat ="identity") + scale_y_continuous(labels = scales::label_number_si()) +
                labs(title = "Traffic for each Year(by passengers)",
                x = "Year", y = "Passengers Count")


So, based on these graphs and data analysis, we can see that at first glance, 2015 is the largest year in terms of passenger traffic and number of flights. We can see that every year the number of passengers and flights is growing, which is logical. But, the question is why 2016 has the least number of passengers? It depends on the fact that it has the least records. Since I have no contact with the person who created this database, I can conclude that some data were simply not added. Let's try to find out.

> SELECT Distinct Month
FROM 
  `air_traffic_passenger.passenger_traffic`
where Year = 2016

If we make such a query to the DB, in my case it was bigQuery, we can see that result will be ["January", "February", "March"]. If we will do the same for 2015 result is all 12 month in year. So, we found a problem, the data for 2016 were taken only for the first 3 months, which does not allow to fully assess and draw conclusions for 2016.

Now let's turn to the year in which the passenger traffic was the lowest. Let's check whether all months were included in 2005. Make the same SQL query, but now where = 2005. 

> SELECT Distinct Month
FROM 
  `air_traffic_passenger.passenger_traffic`
where Year = 2005

We received data only for ["July", "August", "September", "October", "November", "December"]. So, if we do not take this into account, 2005 is the smallest year in terms of passenger traffic and if we do the analysis. But if we look only at the data, then 2005 will be rejected and our option will be 2006 for which data is available for all 12 months.

<a id='ticket-cost'></a>


### Cheap ticket or expensive?


Now let's consider one more question. Let's explore which price category is most suitable for people to plan a trip. Do they choose cheap or expensive tickets, and let's try to find out why. For this question, we need the Price Category Code column, which takes the following values: 

In [ ]:
#showing all types in price_category_code column
passenger_traffic_data %>% 
        drop_na() %>%
        distinct(price_category_code)

It will try to establish links between data, for this we will take the field of region, passengers, date and the price itself

In [ ]:
#counting for each price type
df %>%
    drop_na() %>%
    group_by(price_category_code) %>%
    summarise(adjusted_passenger_count = sum(adjusted_passenger_count))

This data is enough to establish that people choose tickets with the type other more often than low fare. But the question is what other includes, this question should be asked to those who created this database, but I do not have such an opportunity. But still, why people choose other more than low fare, for which regions it is especially and for which month. Let's try to find out

In [ ]:
#getting all data for low fare value and groping by geo_summary, price_category_code
df_lowFare_df <- df %>%
    drop_na() %>%
    group_by(price_category_code, geo_summary) %>%
    filter(price_category_code == 'Low Fare') %>%
    summarise(adjusted_passenger_count = sum(adjusted_passenger_count))
    
#getting all data for other value and groping by geo_summary, price_category_code
df_other_df <- df %>%
    drop_na() %>%
    group_by(price_category_code, geo_summary) %>%
    filter(price_category_code == 'Other') %>%
    summarise(adjusted_passenger_count = sum(adjusted_passenger_count))
print(df_other_df)


piepercent_low<- round(100*df_lowFare_df$adjusted_passenger_count/sum(df_lowFare_df$adjusted_passenger_count), 1)
piepercent_other<- round(100*df_other_df$adjusted_passenger_count/sum(df_other_df$adjusted_passenger_count), 1)

pie(df_lowFare_df$adjusted_passenger_count, labels=paste(df_lowFare_df$geo_summary, "(" , piepercent_low, "%)", sep=""), 
    main = "Flight Type for Low Fare", 
    col = rainbow(length(df_lowFare_df$geo_summary)))
pie(df_other_df$adjusted_passenger_count, labels=paste(df_other_df$geo_summary, "(" , piepercent_other, "%)", sep=""), 
    main = "Flight Type for Other", 
    col = rainbow(length(df_other_df$geo_summary)))

From these two graphs, we can see people choose more cheap ticket types to travel. This can happen for several reasons. The first one is that tickets for traveling within a country are cheaper than for flying between different countries, this is obvious. Another reason is that people don't want to spend a lot of money to fly for a few hours, and spend more for a longer flight with comfort.

Next, let's try to see which regions people choose which tickets. To do this, instead of geo_summary we will use geo_region.

In [ ]:
#getting region and groping by geo_code, price_category_code
df_region <- df %>%
    drop_na() %>%
    group_by(price_category_code, geo_region) %>%
    summarise(adjusted_passenger_count = sum(adjusted_passenger_count))%>%
    arrange(-adjusted_passenger_count)
df_region

ggplot(data=df_region) + geom_point(aes(x=geo_region, 
                y=adjusted_passenger_count, size=adjusted_passenger_count)) + scale_y_continuous(labels = scales::label_number_si()) 
                labs(title = "Traffic for each Year(by passengers)",
                x = "Region", y = "Passengers Count")

From these data we can see that most people travel to the US and most of them choose expensive tickets. It is also easy to see that almost all tickets with low fare type are on the last seats, so people choose other instead of low fare.

So, taking into account these graphs and data, we can conclude that people who choose the type Other are much more than those who choose Low Fare. But we do not know for sure which positions Other includes.

<a id='conclusions'></a>

# Conclusions


This analysis aimed to analyze the flight database from 2005 to 2016, which contains more than 15 thousand records.

This dashboard was intended to explore the database in a certain way, to consider in more detail the flights of people, passenger traffic and to draw certain conclusions based on this.

I would also like to warn that it is impossible to assert the reliability of the statements provided and the statistical validity of the data, but the conclusions obtained as a result of this analysis can be the basis for an in-depth study of these issues.

### Dataset and structure

The dataset consists of 17 columns and more than 15 thousand records. 
* it has a lot of information, namely: information about the year and month traffic, terminals, regions, number of passengers and more 
* the data was partially cleaned, but not completely, some columns had the wrong type 
* there is also a lot of lost data, for example for 2005 and 2016, which include several months, which is not good for analysis
* there is not enough information to study certain problems, for example, pricing policy

### Price Category Code

For this column we have only 2 values *other/low fare*, which did not allow us to investigate the price category in more detail. Some conclusions can be made incorrectly, so you need to take this into account.

### GEO Region

The data is not entered in the correct format because it does not meet the standards. That is, the computer cannot take and break them into certain categories, for example, for the US or America, Mexico everything works well, but for Europe and Asia the situation is more complicated.

### Domestic vs International

While researching this issue, I found some problems with the table. But if we summarize, it turned out that people fly more within the country, such 77 percent, which is 339042566 people. For International the rest, that is 23 percent and 101093590 people. This is a very big difference, but it is logical that people travel more within the country than outside it.

### The most productive year

This issue is not unambiguous, as the database does not fully take into account the years 2005, 2016. If we make certain predictions, we can say that 2016 would have the highest number of arrivals compared to the others. But if we do not take this into account, the best year was 2015, which is the leader. 
If we talk about the worst year, the situation is the same as with the best, we do not have complete information for 2005, so we can not be sure. But if we put this aside, then yes, 2005 is the worst year in terms of passenger traffic.

### Low Fare or Other?

Yes, it looks strange, low price and so on. What else? It is not clear, because other can contain a lot of different price categories. For this column it would be cool to create more useful information, to keep it in a slightly different form. But if we talk about what we have, the situation is that for domestic flights people choose low price, and for long-distance flights - other. This may be due to the fact that long-distance flights are simply more expensive than flights between neighboring cities. Also, do not forget that the flight between neighboring cities can be from one country to another, it is not expensive, and the parameter is not domestic but international. 
Speaking about this table, there are more people who chose an expensive price tag than people who chose an expensive price tag.

### Additional Content

Also, you can check out the Tableau dashboard I created for this project. There are 3 graphs that can be viewed using filters and in a better format. Link: [tableau.cloud](https://prod-uk-a.online.tableau.com/#/site/denysyefimov/workbooks/267615?:origin=card_share_link)